In [1]:
# import datetime
import copy
from datetime import timedelta
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
import random
import os

from pybit import usdt_perpetual
session_unauth = usdt_perpetual.HTTP(endpoint="https://api-testnet.bybit.com")

'''
parameters: token: usdtperp on bybit, 
            interval: 1,5,...,D,W,M , 
            rewind_time: how many days in the past. 
returns pd.DataFrame w/ cols:['symbol','interval','timestamp','date','open','high','low','close']
'''
def create_df(token, interval,rewind_time):
    df = {'symbol':[],'interval':[],'timestamp':[],'date':[],'open':[],'high':[],'low':[],'close':[]}
    curr_ts = int(datetime.now().timestamp())
    start_ts = int((datetime.now() - timedelta(days=rewind_time)).timestamp())

    msg_recv = session_unauth.query_kline(symbol=token, interval = interval, limit = 200, from_time=int(start_ts))

    results = msg_recv['result']
    df = df_filler(df,results)
    last_ts = int(list(df['timestamp'])[-1])

    while curr_ts > last_ts:
        msg_recv = session_unauth.query_kline(symbol=token,interval = interval,limit = 200, from_time=last_ts)
        results = msg_recv['result']
        df = df_filler(df,results)
        last_ts = int(list(df['timestamp'])[-1])

    data = pd.DataFrame(df).drop_duplicates(subset=['date'],keep='first').reset_index(drop=True)
    return data


'''
parameters: old_data: prev. pd.DataFrame w. OHLC data,
            token: should be the same as old_data 
            interval: should be the same as original, 
returns pd.DataFrame w/ cols:['symbol','interval','timestamp','date','open','high','low','close']
'''
def update_df(old_data, token, interval):
    curr_ts = int(datetime.now().timestamp())
    last_ts = old_data.loc[old_data.shape[0]-1,'timestamp']
    last_ts

    df = {'symbol':[],'interval':[],'timestamp':[],'date':[],'open':[],'high':[],'low':[],'close':[]}
    while curr_ts > last_ts:
        msg_recv = session_unauth.query_kline(symbol=token,interval = interval,limit = 200,from_time=last_ts)
        results = msg_recv['result']
        df = df_filler(df,results)
        last_ts = int(list(df['timestamp'])[-1])
    new_data = pd.DataFrame(df).drop_duplicates(subset=['date'],keep='first').reset_index(drop=True)
    ndf = pd.concat([old_data,new_data],join='inner').drop_duplicates(subset=['date'],keep='first').reset_index(drop=True)
    return ndf

# Helper function for above functions
def df_filler(dff,res):
    for i in range(len(res)):
        r = res[i]
        dff['symbol'].append(r['symbol'])
        dff['interval'].append(r['interval'])
        dff['timestamp'].append(r['open_time'])
        dff['date'].append(datetime.strftime(datetime.fromtimestamp(r['open_time']),'%Y-%m-%d %H:%M:%S'))
        dff['open'].append(r['open'])
        dff['high'].append(r['high'])
        dff['low'].append(r['low'])
        dff['close'].append(r['close'])
        # dff['volume'].append(r['volume'])
        # dff['turnover'].append(r['turnover']) 
    return dff

# STAND ALONE TO MEASURE PERP TRADES FOR GNS Platform
def perp_pnl(entry,exit,leverage,side,collateral):
 
    if side == 'Long':
        ls = 1          # 1 if long, -1 if short
    elif side == 'Short':
        ls = -1

    position = np.round(collateral*leverage,2)
    rollover = 0.0063/100
    usdting = 0.0015/100
    liq_price_distance = entry * ( collateral*0.9 - rollover*collateral - usdting*collateral )/collateral/leverage
    liq_price = np.round(entry - ls*liq_price_distance,2)

    pnl_frac  = np.round(ls*leverage*(exit - entry)/entry,3)
    pnl_val = np.round(collateral*pnl_frac,2)
    pnl_perc = np.round(100*(pnl_val/collateral),3)

    return pnl_val,pnl_perc

def code_completed():
    os.system("afplay " + 'sample.wav')